In [1]:
#settings 
from settings.settings import data_settings

#imports
import pandas as pd 
import numpy as np 
# import tensorflow as tf
# import keras
# from keras import layers
# import pathlib
# import datetime
# from model.blocks.keras_tcn import TCN

In [2]:
# Loading data and making it ready
name = data_settings.data_name
csv_path = data_settings.csv_path

train_path = csv_path + "preprocessed/" + name + "/train.csv"
test_path = csv_path + "preprocessed/" + name + "/test.csv"

train_df = pd.read_csv(train_path)

test_df = pd.read_csv(test_path)

test_buy = test_df.pop(test_df.Buy.name)
test_nothing= test_df.pop(test_df.do_nothing.name)
test_sell = test_df.pop(test_df.Sell.name)

train_buy = train_df.pop(train_df.Buy.name)
train_nothing = train_df.pop(train_df.do_nothing.name)
train_sell = train_df.pop(train_df.Sell.name)

train_date_time = train_df.pop(train_df.DateTime.name)
test_date_time = test_df.pop(test_df.DateTime.name)

train_close = train_df.pop("Close")
test_close = test_df.pop("Close")
train_open = train_df.pop("Open")
test_open = test_df.pop("Open")
train_high = train_df.pop("High")
test_high = test_df.pop("High")
train_low = train_df.pop("Low")
test_low = test_df.pop("Low")

relative_columns = ["BOLLINGER_HBAND", "BOLLINGER_LBAND", "BOLLINGER_MAVG",
                    "DONCHIAN_CHANNEL_HBAND", "DONCHIAN_CHANNEL_LBAND", "DONCHIAN_CHANNEL_MBAND",
                    "EMA_INDICATOR_12", "EMA_INDICATOR_26", "EMA_INDICATOR_50",
                    "EMA_INDICATOR_100", "EMA_INDICATOR_200", "SMA_INDICATOR_12",
                    "SMA_INDICATOR_26", "SMA_INDICATOR_50", "SMA_INDICATOR_100","SMA_INDICATOR_200", 
                    # "Open", "High","Low",
                    "ICHIMOKU_A", "ICHIMOKU_B","ICHIMOKU_BASE_LINE", "ICHIMOKU_CONVERSION_LINE", "KELTNER_CHANNEL_HBAND",
                    "KELTNER_CHANNEL_LBAND", "KELTNER_CHANNEL_MBAND", "NEGATIVE_VOLUME_INDEX",
                    "VOLUME_WEIGHTED_AVERAGE_PRICE"]

for column in relative_columns:
    train_df.pop(column)
    test_df.pop(column)


In [ ]:
# Normalizing with z-score https://developers.google.com/machine-learning/data-prep/transform/normalization

# train_df.mean().to_csv(f"{data_settings.base_path}/data/csv/train_mean.csv")
# train_df.std().to_csv(f"{data_settings.base_path}/data/csv/train_std.csv")
# train_df = (train_df - train_df.mean())/train_df.std()
# test_df = (test_df - train_df.mean())/train_df.std()

# Corrilation of buy and sell and do nothing with other rows of dataframes
# pd.options.display.max_rows =1000

# train_df.corr()["buy_4"]
# train_df.corr()["sell_4"]
# train_df.corr()["do_nothing"]

# import matplotlib.pyplot as plt

# for column in df.columns:
    
#     hist = df[column].hist(bins=100)
#     plt.savefig(f"./tmp/{column}_normalized.png")
#     hist.clear()


In [4]:
X_train = (train_df - train_df.mean())/train_df.std()
X_test = (test_df - train_df.mean())/train_df.std()

# X_train = train_df
# X_test = test_df

y_train = []
for buy,nothing,sell in zip(train_buy,train_nothing,train_sell):
    if(buy == 1):
        y_train.append(0)
    elif(nothing == 1):
        y_train.append(1)
    elif(sell == 1):
        y_train.append(2)

y_test = []
for buy,nothing,sell in zip(test_buy,test_nothing,test_sell):
    if(buy == 1):
        y_test.append(0)
    elif(nothing == 1):
        y_test.append(1)
    elif(sell == 1):
        y_test.append(2)     
print(X_train.shape,X_test.shape)


(26164, 93) (1378, 93)


In [5]:
from xgboost import XGBRFClassifier
from sklearn.metrics import log_loss , precision_score
from xgboost.callback import EvaluationMonitor , TrainingCallback

size = 1000
max_depth=8
learning_rate = 0.01
max_bin=2048

model = XGBRFClassifier(
                        n_estimators=size,
                        max_depth=max_depth,
                        max_leaves=0,
                        max_bin=max_bin,
                        grow_policy='depthwise',
                        learning_rate=learning_rate,
                        # verbosity = 2,
                        booster='gbtree',
                        tree_method = "gpu_hist",
                        n_jobs=0,
                        gamma=0,
                        min_child_weight=1,
                        sampling_method='gradient_based',
                        colsample_bytree=1,
                        colsample_bylevel=1,
                        reg_alpha=0,
                        base_score=0.5,
                        random_state=23,
                        missing=np.nan,
                        num_parallel_tree=size,
                        monotone_constraints='()',
                        interaction_constraints='',
                        importance_type=None,
                        gpu_id=0,
                        predictor='auto',
                        enable_categorical=True,
                        max_cat_to_onehot=3,
                        eval_metric = precision_score,
                        # callbacks = [TrainingCallback],
                        # early_stopping_rounds=None,
                        max_delta_step=0,
                        num_class = 3,
                        objective='multi:softmax',
                        )

In [7]:
eval_set = [(X_test, y_test)]

model.fit(
            X_train,
            y_train,
            # eval_set = eval_set,
            verbose = 3,
        )


XGBRFClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                colsample_bylevel=1, colsample_bytree=1,
                early_stopping_rounds=None, enable_categorical=True,
                eval_metric=<function precision_score at 0x0000026043811C10>,
                gamma=0, gpu_id=0, grow_policy='depthwise',
                importance_type=None, interaction_constraints='',
                learning_rate=0.01, max_bin=2048, max_cat_to_onehot=3,
                max_delta_step=0, max_depth=8, max_leaves=0, min_child_weight=1,
                missing=nan, monotone_constraints='()', n_estimators=1000,
                n_jobs=0, num_class=3, num_parallel_tree=1000,
                objective='multi:softmax', predictor='auto', random_state=23,
                reg_alpha=0, ...)

In [8]:
p_train = model.predict(X_train)
p_test = model.predict(X_test)

In [10]:
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report
import pathlib

acc_train = accuracy_score(p_train,y_train)
acc_test = accuracy_score(p_test,y_test)

rep_train = classification_report(y_train,p_train)
rep_test = classification_report(y_test,p_test)

con_train = confusion_matrix(y_train,p_train)
con_test = confusion_matrix(y_test,p_test)


# #check if path is available
# path = f'./model/models/XGBOOST/model-{size}-{max_depth}-{learning_rate}-{max_bin}'

# pathlib.Path(path).mkdir(parents=True,exist_ok=True)
# model.save_model(f"{path}/model.json")

# with open(f"{path}/train_report.txt","w") as f:
#     f.write(rep_train)

# with open(f"{path}/test_report.txt","w") as f:
#     f.write(rep_test) 
    
print(rep_train)
print(rep_test)


              precision    recall  f1-score   support

           0       1.00      0.13      0.24      2023
           1       0.85      1.00      0.92     21721
           2       1.00      0.14      0.25      2420

    accuracy                           0.85     26164
   macro avg       0.95      0.43      0.47     26164
weighted avg       0.88      0.85      0.80     26164

              precision    recall  f1-score   support

           0       0.67      0.07      0.13       113
           1       0.81      0.98      0.89      1110
           2       0.11      0.02      0.03       155

    accuracy                           0.80      1378
   macro avg       0.53      0.36      0.35      1378
weighted avg       0.72      0.80      0.73      1378



In [ ]:
# from xgboost import plot_importance ,plot_tree
# import matplotlib as mpl

# # model = XGBRFClassifier()
# # model.load_model(f"./model/models/XGBOOST/model_1000_12/model.json")

# mpl.rcParams['axes.grid'] = False
# mpl.rcParams['figure.figsize'] = (200, 100)

# plot_tree(model)


In [11]:
df = X_test.copy()
df = df * train_df.std() + train_df.mean()
df["DateTime"]= pd.to_datetime(test_date_time)
df=df.set_index(pd.DatetimeIndex(df["DateTime"]))


In [15]:
df["Signal"] = p_test
df["Close"] = test_close.tolist()
df["High"] = test_high.tolist()
df["Open"] = test_open.tolist()
df["Low"] = test_low.tolist()
df[df.Signal==0].shape


(12, 99)

In [49]:
from backtesting import Strategy

def Signal():
    return df.Signal

class MyStrategy(Strategy):
    initsize = 0.04
    mysize = initsize
    c = 0
    def init(self):
        super().init()
        self.signal1 = self.I(Signal)

    def next(self):
        super().next()
        
        if(self.signal1 != 1 and len(self.trades)==0 and len(self.closed_trades)>0 and self.closed_trades[-1].pl < 0):
            if self.mysize != .32:
                self.mysize=self.mysize * 2
            else:
                self.c +=1
            
        elif len(self.closed_trades)>0 and self.closed_trades[-1].pl > 0:
            if self.c > 0:
                # self.mysize=self.mysize / 2
                self.c -=1
            else:
                self.mysize=self.initsize

            
        if self.signal1==0 and self.data.EMA_INDICATOR_100_C >= 1 :#and 
            sl1 = self.data.Close[-1] -self.data.AVERAGE_TRUE_RANGE[-1] * 3
            tp1 = self.data.Close[-1] + 12* self.data.AVERAGE_TRUE_RANGE[-1]
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
            
        if self.signal1==2 and self.data.EMA_INDICATOR_100_C <= 1:# 
            sl1 = self.data.Close[-1] + (self.data.AVERAGE_TRUE_RANGE[-1]*3)
            tp1 = self.data.Close[-1] - (12 * self.data.AVERAGE_TRUE_RANGE[-1])
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

In [53]:
from backtesting import Backtest

bt = Backtest(df, MyStrategy, cash=2_000, margin=1/100, commission=.001,)
stat = bt.run()
stat



Start                     2022-03-30 20:00:00
End                       2022-06-20 05:00:00
Duration                     81 days 09:00:00
Exposure Time [%]                   23.149492
Equity Final [$]                  5528.016262
Equity Peak [$]                   5528.016262
Return [%]                         176.400813
Buy & Hold Return [%]               -5.654572
Return (Ann.) [%]                 7589.475909
Volatility (Ann.) [%]          1440908.527584
Sharpe Ratio                         0.005267
Sortino Ratio                       79.037934
Calmar Ratio                       169.402859
Max. Drawdown [%]                  -44.801345
Avg. Drawdown [%]                  -10.610688
Max. Drawdown Duration       12 days 05:00:00
Avg. Drawdown Duration        1 days 22:00:00
# Trades                                   12
Win Rate [%]                        66.666667
Best Trade [%]                       2.172376
Worst Trade [%]                     -0.762731
Avg. Trade [%]                    

In [95]:
bt.plot()

Row(id='4585', ...)

In [54]:
from lightgbm import LGBMClassifier


In [ ]:
# Creating generators

# window_size = data_settings.window_size
# batch_size : int = 256
# num_features : int = 97
# num_classes : int = 3

# def x_train_generator():
#     for i in range(window_size,len(train_df)+1):
#         window=train_df[i-window_size:i]
#         yield window.to_numpy()

# def x_test_generator():
#     for i in range(window_size,len(test_df)+1):
#         window=test_df[i-window_size:i]
#         yield window.to_numpy()    
        
# def y_train_generator():
#     for i in range(window_size,train_labels.shape[0]+1):
#         yield train_labels[i-1]

# def y_test_generator():
#     for i in range(window_size,test_labels.shape[0]+1):
#         yield test_labels[i-1]
        
# x_train_ds = tf.data.Dataset.from_generator(x_train_generator,
#                                         output_signature=(
#                                             tf.TensorSpec(shape=(window_size,num_features), dtype=tf.float64)
#                                             )
#                                         ).batch(batch_size)

# x_test_ds = tf.data.Dataset.from_generator(x_test_generator,
#                                         output_signature=(
#                                             tf.TensorSpec(shape=(window_size,num_features), dtype=tf.float64)
#                                             )
#                                         ).batch(batch_size)
# y_train_ds = tf.data.Dataset.from_generator(y_train_generator,
#                                         output_signature=(
#                                             tf.TensorSpec(shape=(num_classes), dtype=tf.float32)
#                                             )
#                                         ).batch(batch_size)
# y_test_ds = tf.data.Dataset.from_generator(y_test_generator,
#                                         output_signature=(
#                                             tf.TensorSpec(shape=(num_classes), dtype=tf.float32)
#                                             )
#                                         ).batch(batch_size)

# train_ds = tf.data.Dataset.zip((x_train_ds,y_train_ds))
# test_ds = tf.data.Dataset.zip((x_test_ds,y_test_ds))

In [ ]:
# def compile_and_fit(model:keras.Model,modelname,train_ds=train_ds,test_ds=test_ds,modelcheckpoint:bool = False,epochs= 10):
#     import os

#     #check if path is available
#     path = f'./model/models/{modelname}/{data_settings.stockname}/tensorboard/logs/fit'
#     pathlib.Path(path).mkdir(parents=True,exist_ok=True)

#     #tensorboard
#     log_dir =f'./model/models/{modelname}/{data_settings.stockname}/tensorboard/logs/fit/{datetime.datetime.now().strftime("(%Y-%m-%d)-(%H-%M-%S)")}'
#     tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#     #modelcheckpoint
#     base_path = f"./model/models/{modelname}/"
#     checkpoint_path = base_path + "tmp/cp-{epoch:04d}.ckpt"
#     checkpoint_dir = os.path.dirname(checkpoint_path)

#     model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
#         filepath=checkpoint_path,
#         monitor='val_loss',
#         verbose=1,
#         save_weights_only=True,
#         save_best_only=False
#     )

#     model.save_weights(checkpoint_path.format(epoch=0))


#     model.compile(loss='categorical_crossentropy',
#                     optimizer='adam',
#                     metrics=["categorical_accuracy","categorical_crossentropy"],
#                 )

#     if modelcheckpoint:
#         history = model.fit(
#                             train_ds,
#                             validation_data= test_ds,
#                             shuffle=True,
#                             epochs=epochs,
#                             verbose=1,
#                             callbacks=[tensorboard_callback,model_checkpoint_callback],
#                             use_multiprocessing=True,
#                             )
#     else:
#         history = model.fit(    
#                             train_ds,
#                             validation_data=test_ds,
#                             shuffle=True,
#                             epochs=epochs,
#                             verbose=1,
#                             callbacks=[tensorboard_callback],
#                             use_multiprocessing=True,
#                            )
    
#     return history

In [ ]:
#TF Model
# input_layer = layers.Input(shape=(window_size,num_features))
# x = TCN(
#     nb_filters=128,
#     # kernel_size=3,
#     # nb_stacks=1,
#     # padding="causal",
#     # activation="swish",
#     use_batch_norm=True,
#     # return_sequences=True
#     )(input_layer)

# # x = layers.Flatten()(x)
# x = layers.Dense(3,activation="softmax")(x)


# model = keras.Model(
#     inputs = input_layer,
#     outputs = x
# )

# input_layer = layers.Input(shape=(window_size,num_features))

# x = layers.Dense(1024,activation="relu")(input_layer)
# x = layers.Dense(512,activation="relu")(x)
# x = layers.Dense(256,activation="relu")(x)
# x = layers.Dense(128,activation="relu")(x)
# x = layers.Dense(64,activation="relu")(x)
# x = layers.Dense(32,activation="relu")(x)
# x = layers.Dense(16,activation="relu")(x)
# x = layers.Flatten()(x)
# x = layers.Dense(3,activation="softmax")(x)


# model = keras.Model(
#     inputs = input_layer,
#     outputs = x
# )
# model.summary()
# compile_and_fit(
#     model=model,
#     modelname="simple_dense_first_try",
#     train_ds=train_ds,
#     test_ds=test_ds,
#     modelcheckpoint=True
# )